In [ ]:
from google.colab import drive
drive.mount('content')

Mounted at content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/MyDrive/New Work/MRI-classifier/custom_data_3D")

In [ ]:
!pip install antspyx
!pip install SimpleITK
!pip install antspynet

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact

# MRI related
import ants
from antspynet.utilities import brain_extraction
import SimpleITK as sitk

In [ ]:
# Load template image
template_img_path = "template.nii"
template_img = ants.image_read(template_img_path, reorient='IAL')

In [ ]:
def explore_3D_array(arr: np.ndarray, cmap: str = 'gray'):
  def fn(SLICE):
    plt.figure(figsize=(7,7))
    plt.imshow(arr[SLICE, :, :], cmap=cmap)

  interact(fn, SLICE=(0, arr.shape[0]-1))

In [ ]:
# create output save folder
processed_img_folder_path = "processed_images"
class_name = "MCI"
processed_img_folder_path = os.path.join(processed_img_folder_path, class_name)

# subfolders
affine_registration_img_folder = "affine_registration"
bias_correction_img_folder = "bias_correction"
skull_stripping_img_folder= "skull_stripping"

# subfolders paths
affine_registration_img_folder_path = os.path.join(processed_img_folder_path,
                                                   affine_registration_img_folder)
bias_correction_img_folder_path = os.path.join(processed_img_folder_path,
                                                   bias_correction_img_folder)
skull_stripping_img_folder_path = os.path.join(processed_img_folder_path,
                                                   skull_stripping_img_folder)

# create output folders
folder_path_list = [affine_registration_img_folder_path,
                    bias_correction_img_folder_path,
                    skull_stripping_img_folder_path,
                    ]

for subfolder_path in folder_path_list:
  os.makedirs(subfolder_path, exist_ok = True)

# Affine Registration

In [ ]:
def affine_registration(input_img, template_img):
  transformation = ants.registration(
      fixed=template_img,
      moving=input_img,
      type_of_transform='SyN',
      verbose=True
  )
  registered_img_ants = transformation['warpedmovout']
  return registered_img_ants

In [ ]:
# for test visualization
# explore_3D_array(arr=img.numpy())
# explore_3D_array(arr=source_image.numpy())

# Bias correction

In [ ]:
def explore_3D_array_bias(arr: np.ndarray, cmap: str = 'gray'):
  def fn(SLICE):
    plt.figure(figsize=(7,7))
    plt.imshow(arr[SLICE, :, :], cmap=cmap)

  interact(fn, SLICE=(0, arr.shape[0]-1))

In [ ]:
def bias_correction(registered_img_path):
  # load image
  raw_img_sitk = sitk.ReadImage(registered_img_path, sitk.sitkFloat32)
  raw_img_sitk = sitk.DICOMOrient(raw_img_sitk,'RPS')
  raw_img_sitk_arr = sitk.GetArrayFromImage(raw_img_sitk)

  # create head mask
  transformed = sitk.RescaleIntensity(raw_img_sitk, 0, 255)
  #transformed = sitk.TriangleThreshold(transformed, 0, 1)
  transformed = sitk.LiThreshold(transformed,0,1)
  head_mask = transformed

  # bias correction
  shrinkFactor = 4
  inputImage = raw_img_sitk
  inputImage = sitk.Shrink( raw_img_sitk, [ shrinkFactor ] * inputImage.GetDimension() )
  maskImage = sitk.Shrink( head_mask, [ shrinkFactor ] * inputImage.GetDimension() )
  bias_corrector = sitk.N4BiasFieldCorrectionImageFilter()
  corrected = bias_corrector.Execute(inputImage, maskImage)

  # get image corrected
  log_bias_field = bias_corrector.GetLogBiasFieldAsImage(raw_img_sitk)
  corrected_image_full_resolution = raw_img_sitk / sitk.Exp(log_bias_field)

  return corrected_image_full_resolution

# Skull Stripping

In [ ]:
def rescale_linear(array: np.ndarray, new_min: int, new_max: int):
  """Rescale an array linearly."""
  minimum, maximum = np.min(array), np.max(array)
  m = (new_max - new_min) / (maximum - minimum)
  b = new_min - m * minimum
  return m * array + b

In [ ]:
def skull_stripping(bias_corrected_image):
  prob_brain_mask = brain_extraction(bias_corrected_image, modality = "t1", verbose=True)
  brain_mask = ants.get_mask(prob_brain_mask, low_thresh=0.5)
  masked = ants.mask_image(bias_corrected_image, brain_mask)
  return masked

# Final loop

In [ ]:
# main processing loop
source_folder = "custom_data_3D"
source_img_folder_path = os.path.join(source_folder, class_name)
source_img_list = os.listdir(source_img_folder_path)

for source_img_name in source_img_list:
  source_image_path = os.path.join(source_img_folder_path, source_img_name)
  source_image = ants.image_read(source_image_path, reorient='IAL')

  # --------------------- perform affine registration --------------------------
  registered_img_ants = affine_registration(input_img = source_image,
                                            template_img = template_img)
  # save registered image
  registered_img_path = os.path.join(affine_registration_img_folder_path,
                                  source_img_name)
  registered_img_ants = sitk.GetImageFromArray(registered_img_ants.numpy())
  sitk.WriteImage(registered_img_ants, registered_img_path)

  # --------------------- perform bias correction ------------------------------
  bias_corrected_image = bias_correction(registered_img_path)

  # save bias corrected image
  bias_corrected_img_path = os.path.join(bias_correction_img_folder_path,
                                  source_img_name)
  sitk.WriteImage(bias_corrected_image, bias_corrected_img_path)

  # ----------------------- perform skull stripping ----------------------------
  skull_stripping_input_img = ants.image_read(bias_corrected_img_path,
                                            reorient='IAL')
  skull_stripped_img = skull_stripping(bias_corrected_image =
                                      skull_stripping_input_img)
  skull_stripped_img = sitk.GetImageFromArray(skull_stripped_img.numpy())

  # save the skull stripped image
  skull_stripped_img_path = os.path.join(skull_stripping_img_folder_path,
                                        source_img_name)
  sitk.WriteImage(skull_stripped_img, skull_stripped_img_path)

antsRegistration -d 3 -r [0x7a0699530968,0x7a069d42ad48,1] -m mattes[0x7a0699530968,0x7a069d42ad48,1,32,regular,0.2] -t Affine[0.25] -c 2100x1200x1200x0 -s 3x2x1x0 -f 4x2x2x1 -x [NA,NA] -m mattes[0x7a0699530968,0x7a069d42ad48,1,32] -t SyN[0.200000,3.000000,0.000000] -c [40x20x0,1e-7,8] -s 2x1x0 -f 4x2x1 -u 1 -z 1 -o [/tmp/tmply_65y8i,0x7a069de197a8,0x7a069d429e28] -x [NA,NA] --float 1 --write-composite-transform 0 -v 1
Brain extraction:  retrieving model weights.
Brain extraction:  retrieving template.
Brain extraction:  normalizing image to the template.
Brain extraction:  prediction and decoding.
1/1 ━━━━━━━━━━━━━━━━━━━━ 22s 22s/step
Brain extraction:  renormalize probability mask to native space.
antsRegistration -d 3 -r [0x7a069dc034e8,0x7a069dc03968,1] -m mattes[0x7a069dc034e8,0x7a069dc03968,1,32,regular,0.2] -t Affine[0.25] -c 2100x1200x1200x0 -s 3x2x1x0 -f 4x2x2x1 -x [NA,NA] -m mattes[0x7a069dc034e8,0x7a069dc03968,1,32] -t SyN[0.200000,3.000000,0.000000] -c [40x20x0,1e-7,8] -s 2